# 文書生成AIのファインチューニング・レシピ
本Notebookでは、一対比較によって訓練した報酬モデルを用いたRLHFにより事前学習済みモデルをファインチューニングするコードを記載します。

## 応用レシピ：独自の報酬モデルによる事前学習済みモデルのファインチューニング

### (1) Hugging Face HubとWeights & Biasesのアカウント作成
これらのアカウントは作成済みであるとします。

### (2) ライブラリのインストール

利用するライブラリをインストールします。

In [1]:
%pip install transformers==4.35.2
%pip install trl[peft]==0.7.10
%pip install wandb==0.16.2
%pip install sentencepiece==0.1.99
%pip install accelerate==0.26.1
%pip install bitsandbytes==0.42.0
%pip install datasets==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━

### (3) Weights & Biases及びHugging Face Hubへのログイン
学習時のログを記録するWeights & Biases、及び訓練後のモデルの保存先であるHuggingface Hubにログインします。いずれも事前にアカウントの作成とトークンの取得が必要です。

In [2]:
import wandb
wandb.init()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from huggingface_hub import notebook_login
notebook_login()

ライブラリをインポートします。

In [4]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from peft import LoraConfig
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


### (4) データセットの準備

モデル名とデータセット名を定義します。

In [5]:
base_model_name_for_reward = 'rinna/japanese-gpt2-medium'
reward_model_name = 'taku-yoshioka/reward-model-0828'
model_name = 'line-corporation/japanese-large-lm-1.7b'
dataset_name, subset_name = 'shunk031/JGLUE', 'MARC-ja'
save_model_name = 'taku-yoshioka/rlhf-llm-custom-rm-0828'

データセットを作成する関数を定義します。

In [6]:
def build_dataset(model_name, dataset_name, subset_name=None, split="train",
                  input_min_text_length=2, input_max_text_length=8):
    # トーカナイザをロード
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # データセットをロード、長さが200以上のサンプルを残す
    ds = load_dataset(dataset_name, split=split, name=subset_name,
                      trust_remote_code=True)
    ds = ds.rename_columns({"sentence": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    # 各サンプルの長さをランダムに選ぶ
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    # 入力サンプルに対して長さをランダムに選び、トークン列に変換
    def tokenize(sample):
        sample["input_ids"] = \
            tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [7]:
# データセットを取得し、RLHFの対象となるモデルのトーカナイザでトークン列に変換
dataset = build_dataset(model_name, dataset_name, subset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


/root/.cache/huggingface/modules/datasets_modules/datasets/shunk031--JGLUE/bef660b0971cf102c95d2554bd2552ce5fb1cf8e9223cc6f8001c70be571c18c/JGLUE.py:464: UserWarning: can't import `mojimoji`, failing back to method that do nothing. We recommend running `pip install mojimoji` to reproduce the original preprocessing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/187528 [00:00<?, ? examples/s]

Map:   0%|          | 0/71819 [00:00<?, ? examples/s]

### (5) 事前学習済みモデルの読み込み

Hugging Face Hub上の事前学習済みモデルを読み込みます。

In [8]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    peft_config=lora_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/trl/models/modeling_base.py:326: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = loading_func(filename if not use_safe els

### (6) 報酬モデルの読み込み

報酬モデルを読み込みます。

In [9]:
device = 0
sentiment_pipe = pipeline("sentiment-analysis",
                           model=AutoModelForSequenceClassification.from_pretrained(reward_model_name, num_labels=1),
                           tokenizer=AutoTokenizer.from_pretrained(base_model_name_for_reward),
                           device=device)
sentiment_pipe.model.config.pad_token_id = tokenizer.eos_token_id

adapter_config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at rinna/japanese-gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

In [10]:
sentiment_pipe("これは美味しくない", return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 0.0042572179809212685}]]

## ファインチューニング実装

### (1) 学習条件の設定
学習条件の設定を`PPOTrainer`のコンストラクタに渡してインスタンス化します。

In [11]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

config = config = PPOConfig(
    model_name,
    learning_rate=1.41e-5,
    log_with="wandb",
)
ppo_trainer = PPOTrainer(config, model, tokenizer=tokenizer, dataset=dataset, data_collator=collator)

### (2) ファインチューニングの実行

以下のコードでファインチューニングを実行します。

In [12]:
# 感情分析パイプラインのパラメータ
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

# 生成文の長さをランダムに選択
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

# 文書生成パイプラインのパラメータ
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    # # 動作確認のための小さめのステップ数
    # if step == 1:
    #    break

    # ステップ数が大きいとKLダイバージェンスの項が増大していくので学習を打ち切ります
    if step == 50:
        break

    # 文書生成
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # 報酬（感情スコア）の計算
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)

    assert pipe_outputs[0][0]["label"] == "LABEL_0"
    rewards = [torch.tensor(output[0]["score"]) for output in pipe_outputs]

    # PPO訓練ステップ
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
6it [13:31, 135.85s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (62.17) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (577.48) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (729.80) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
9it [20:18, 135.85s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize effici

### (3) モデルの保存
ファインチューニング済みのモデルを保存します。

In [13]:
ppo_trainer.save_pretrained(save_model_name, push_to_hub=True)

pytorch_model.bin:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/taku-yoshioka/rlhf-llm-custom-rm-0828/commit/ed5992c46f95bc44d0a944e2b5ddeb407965f6a8', commit_message='Push model using huggingface_hub.', commit_description='', oid='ed5992c46f95bc44d0a944e2b5ddeb407965f6a8', pr_url=None, pr_revision=None, pr_num=None)